In [5]:
import ifcopenshell
from topologicpy.Vertex import Vertex
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Plotly import Plotly

# Load the IFC file
ifc_file = ifcopenshell.open("Hus28_test.ifc")
print("IFC file opened successfully.")

# Find the IfcSpace with the name '926'
target_space = None
print("Searching for IfcSpace with Name = '926'...")
for space in ifc_file.by_type("IfcSpace"):
    if space.Name == "926":
        target_space = space
        print("Found target IfcSpace.")
        break

# If the target space is found, navigate to the coordinates
if target_space:
    print(f"Extracting outline coordinates for IfcSpace with Name = 926...")
    placement = target_space.ObjectPlacement
    representation = target_space.Representation

    # Find the IFCCartesianPointList2D through the linked structure
    coordinates = []
    if representation:
        shape = representation.Representations[0]
        if shape and shape.Items:
            for item in shape.Items:
                if item.is_a("IfcExtrudedAreaSolid"):
                    profile = item.SweptArea
                    if profile and profile.is_a("IfcArbitraryClosedProfileDef"):
                        curve = profile.OuterCurve
                        if curve and curve.is_a("IfcIndexedPolyCurve"):
                            point_list = curve.Points
                            if point_list and point_list.is_a("IfcCartesianPointList2D"):
                                coordinates = point_list.CoordList
                                print("Outline coordinates (x, y):")
                                for point in coordinates:
                                    print(f"{point}")
                            else:
                                print("No IfcCartesianPointList2D found.")
                        else:
                            print("No IfcIndexedPolyCurve found.")
                    else:
                        print("No IfcArbitraryClosedProfileDef found.")
                else:
                    print("No IfcExtrudedAreaSolid found.")
    else:
        print("No Representation found for the target IfcSpace.")

    # Convert coordinates to Vertex objects
    vertices = [Vertex.ByCoordinates(x, y, 0) for x, y in coordinates]

    # Create a Wire from the vertices
    wire = Wire.ByVertices(vertices, close=True)

    # Create a Face from the Wire
    face = Face.ByWire(wire)

    # Visualize using Plotly
    data = Plotly.DataByTopology(face)
    figure = Plotly.FigureByData(data)
    Plotly.Show(figure)

else:
    print("No IfcSpace with Name = 926 found.")


IFC file opened successfully.
Searching for IfcSpace with Name = '926'...
Found target IfcSpace.
Extracting outline coordinates for IfcSpace with Name = 926...
Outline coordinates (x, y):
(-1584.5833333332303, 1017.4999999994851)
(-1584.5833333332348, 366.9999999994684)
(-2127.0833333336054, 366.9999999994722)
(-2127.083333333626, -2622.999999999935)
(1545.4166666667606, -2622.999999999952)
(1545.4166666667606, -2616.499999999511)
(1545.4166666667843, 784.4999999994818)
(1545.4166666667859, 1017.5000000005896)
(945.4166666667144, 1017.5000000005938)
(945.4166666667152, 1137.0000000004627)
(-314.5833333336048, 1137.0000000004627)
(-334.58333333323725, 1017.4999999994851)
(-1584.5833333332303, 1017.4999999994851)
